## Explore Climate Change Projections with an Emulator

This notebook creates a simple interactive widget for exploring emulator predictions. Run the notebook to gain access to the emulator, or if you want to understand the details check the `simple_example` notebook first.

In [1]:
import sys
import os
home = os.getenv("HOME")
sys.path.insert(0, '../')
sys.path.insert(0, '../setup/')
sys.path.insert(0, '../plotting/')

import GPy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import sklearn.preprocessing as preprocessing
import sklearn.model_selection as model_selection
from sklearn.decomposition import PCA
import pickle

from plotmapfunction import *
from read_file import *
from AreaWeighting import *
from RegionLatitudes import *
from DefineRegions import *
from conv_MMR_ppm import *


In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Layout

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
%matplotlib inline

In [5]:
### GET FILES ####
# Open datasets
X, Y, Xtest, Ytest, latitude, longitude = get_train_test()


nlat, nlon = len(latitude), len(longitude)
p = X.shape[1]

Getting inputs and outputs
Opening ../../emulator_files/AllTemps1-86.nc
Getting inputs and outputs
Opening ../../emulator_files/TestTemps1-18.nc


In [6]:
## Reshape arrays: flatten output along lon/lat
Xfull, Yfull = X, Y  
Y = Yfull.reshape((Yfull.shape[0], nlon*nlat))
print(Y.shape)


# Scale X and Y to zero mean, unit variance
scalerX = preprocessing.StandardScaler()
scalerX.fit(X)
X = scalerX.transform(X)
Xtest = scalerX.transform(Xtest)

scalerY = preprocessing.StandardScaler()
scalerY.fit(Y)
Y = scalerY.transform(Y)


(86, 27648)


In [7]:
## Kernel
kern = GPy.kern.RBF(p, ARD=True) + GPy.kern.Linear(p, ARD=True)

print("kernel used: {}".format(kern))

m = GPy.models.GPRegression(X, Y, kern)
m.likelihood.variance.fix(0.41)
print(f"Model: {m}")

m.optimize()

print(f"Optimised model: {m}")

kernel used:   sum.              |  value  |  constraints  |  priors
  rbf.variance      |    1.0  |      +ve      |        
  rbf.lengthscale   |   (9,)  |      +ve      |        
  linear.variances  |   (9,)  |      +ve      |        
Model: 
Name : GP regression
Objective : 3254156.3817916904
Number of Parameters : 20
Number of Optimization Parameters : 19
Updates : True
Parameters:
  GP_regression.           |  value  |  constraints  |  priors
  sum.rbf.variance         |    1.0  |      +ve      |        
  sum.rbf.lengthscale      |   (9,)  |      +ve      |        
  sum.linear.variances     |   (9,)  |      +ve      |        
  Gaussian_noise.variance  |   0.41  |   +ve fixed   |        
Optimised model: 
Name : GP regression
Objective : 2093943.8731012486
Number of Parameters : 20
Number of Optimization Parameters : 19
Updates : True
Parameters:
  GP_regression.           |                  value  |  constraints  |  priors
  sum.rbf.variance         |  9.873874988025384e-09  | 

In [8]:
X_CO2     = 410       # ppm between 202 and 1088  (today = 410)
X_CH4     = 1800      # ppb between 247 and 3238  (today = 1850)
X_SO4_Eur = 1.0       # fraction between 0 and 10 (5)
X_SO4_NAm = 1.0       # fraction between 0 and 6  (3)
X_SO4_EAs = 1.0       # fraction between 0 and 4  (2)
X_SO4_SAs = 1.0       # fraction between 0 and 6  (3)
X_SO4_SAm = 1.0       # fraction between 0 and 6  (3)
X_SO4_Afr = 1.0       # fraction between 0 and 14 (7)
X_OCBC_Tr = 1.0       # fraction between 0 and 4  (2)


In [9]:
X_new = np.array([ CO2_ppm_MMR(X_CO2), CH4_ppb_MMR(X_CH4), X_SO4_Eur, X_SO4_NAm, 
                   X_SO4_EAs, X_SO4_SAs, X_SO4_SAm, X_SO4_Afr, X_OCBC_Tr])
X_new

array([6.22968023e-04, 9.96965240e-07, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00])

In [10]:
def get_map(co2_widget, ch4_widget, so2_eur_widget, so2_nam_widget, so2_eas_widget, 
              so2_sas_widget, so2_sam_widget, so2_afr_widget, ocbc_tr_widget ):
    print("CO2:",co2_widget, "ppm. CH4:", ch4_widget, " ppb. \nSO2_EUR:", so2_eur_widget, "x.  SO2_NAm:", so2_nam_widget, 
          "x.  SO2_EAs:", so2_eas_widget, "x.  SO2_SAs:", so2_sas_widget, "x. SO2_SAm:", so2_sam_widget,
          "x.  SO2_Afr:",  so2_afr_widget, "x.  OCBC_Tro:", ocbc_tr_widget, "x. ")
    X_new = np.array([ CO2_ppm_MMR(co2_widget), CH4_ppb_MMR(ch4_widget), so2_eur_widget, so2_nam_widget, so2_eas_widget, 
                      so2_sas_widget, so2_sam_widget, so2_afr_widget, ocbc_tr_widget ])
    X_new_scaled = scalerX.transform(X_new.reshape(1, -1))
    ypred, var = m.predict_noiseless(X_new_scaled)
    ypred = scalerY.inverse_transform(ypred)
    levels = np.arange(-2., 2.01, 0.1)
    ypredmap = ypred.reshape((nlat, nlon))
    plotmap(longitude, latitude, ypredmap, levels=levels, variable_label='Temperature Response ($\degree$C)') 
    std = np.sqrt((scalerY.var_)*(var))
    std = std[0].reshape((nlat, nlon))
    print("Estimated global mean response:{} deg C".format(ypredmap.mean()))
    print("Estimated global mean GP error:{} deg C".format(std.mean()))
    return ypredmap
    

In [11]:
Widget = interactive(get_map, {'manual': True},
                co2_widget=widgets.IntSlider(min=200, max=834, step=10, value=410, description='CO2 (ppm)', layout=Layout(width='50%'), style = {'description_width': '150px'}),
                ch4_widget=widgets.IntSlider(min=250, max=3200, step=10, value=1850, description='CH4 (ppb)', layout=Layout(width='50%'), style = {'description_width': '150px'}),
                so2_eur_widget=widgets.FloatSlider(min=0, max=5, step=0.1, value=1, description='SO2 Europe (frac)', layout=Layout(width='50%'), style = {'description_width': '150px'}),
                so2_nam_widget=widgets.FloatSlider(min=0, max=3, step=0.1, value=1, description='SO2 N. America (frac)', layout=Layout(width='50%'), style = {'description_width': '150px'}),
                so2_eas_widget=widgets.FloatSlider(min=0, max=2, step=0.1, value=1, description='SO2 E. Asia (frac)', layout=Layout(width='50%'), style = {'description_width': '150px'}),
                so2_sas_widget=widgets.FloatSlider(min=0, max=3, step=0.1, value=1, description='SO2 S. Asia (frac)', layout=Layout(width='50%'), style = {'description_width': '150px'}),
                so2_sam_widget=widgets.FloatSlider(min=0, max=3, step=0.1, value=1, description='SO2 S. America (frac)', layout=Layout(width='50%'), style = {'description_width': '150px'}),
                so2_afr_widget=widgets.FloatSlider(min=0, max=7, step=0.1, value=1, description='SO2 Africa (frac)', layout=Layout(width='50%'), style = {'description_width': '150px'}),
                ocbc_tr_widget=widgets.FloatSlider(min=0, max=2, step=0.1, value=1, description='OC/BC Tropics (frac)', layout=Layout(width='50%'), style = {'description_width': '150px'}));

In [12]:
display(Widget);

interactive(children=(IntSlider(value=410, description='CO2 (ppm)', layout=Layout(width='50%'), max=834, min=2…